# Análisis Exploratorio de Imágenes

**Universidad Nacional de Colombia**  
**Visión por Computador**  
**Trabajo 2: Registro de Imágenes**

---

## Objetivos

1. Cargar y visualizar las imágenes del comedor
2. Analizar características básicas (dimensiones, histogramas, etc.)
3. Explorar diferentes detectores de características
4. Comparar métodos de emparejamiento
5. Identificar el mejor enfoque para el registro

In [ ]:
# Importar librerías
import sys
sys.path.append('../src')

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Importar módulos del proyecto
from feature_detection import detect_sift_features, detect_orb_features, detect_akaze_features
from feature_detection import compare_detectors, visualize_keypoints
from matching import match_features, compute_match_statistics, visualize_matches
from utils import load_images_from_directory

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (15, 10)
sns.set_style('whitegrid')

print("✓ Librerías importadas correctamente")

## 1. Carga de Imágenes

Cargamos las imágenes del comedor que vamos a registrar.

In [ ]:
# Directorio de imágenes
data_dir = Path('../data/original')

# Cargar imágenes
images = load_images_from_directory(str(data_dir))

print(f"\nTotal de imágenes cargadas: {len(images)}")
print("\nDetalles:")
for i, (name, img) in enumerate(images, 1):
    print(f"  {i}. {name}: {img.shape[1]}x{img.shape[0]} píxeles")

## 2. Visualización de Imágenes

In [ ]:
# Visualizar todas las imágenes
n_images = len(images)
cols = 2
rows = (n_images + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
axes = axes.flatten() if n_images > 1 else [axes]

for idx, (name, img) in enumerate(images):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'{name}\n{img.shape[1]}x{img.shape[0]}', fontsize=12)
    axes[idx].axis('off')

# Ocultar ejes vacíos
for idx in range(n_images, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('../results/figures/01_images_overview.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Análisis de Histogramas

Analizamos la distribución de intensidades en cada canal de color.

In [ ]:
def plot_histogram(img, title):
    """Muestra el histograma RGB de una imagen"""
    colors = ('b', 'g', 'r')
    plt.figure(figsize=(10, 4))
    
    for i, color in enumerate(colors):
        hist = cv2.calcHist([img], [i], None, [256], [0, 256])
        plt.plot(hist, color=color, label=f'Canal {color.upper()}')
    
    plt.xlim([0, 256])
    plt.xlabel('Intensidad de píxel')
    plt.ylabel('Frecuencia')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Mostrar histogramas
for name, img in images[:2]:  # Primeras 2 imágenes
    plot_histogram(img, f'Histograma RGB - {name}')

## 4. Detección de Características

Comparamos diferentes detectores de características en las imágenes.

In [ ]:
# Seleccionar la primera imagen para análisis
if len(images) > 0:
    test_name, test_img = images[0]
    
    print(f"\nAnalizando: {test_name}")
    print("="*60)
    
    # Comparar detectores
    results = compare_detectors(test_img, detectors=['sift', 'orb', 'akaze'])
    
    # Visualizar resultados
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, (detector_name, (kp, desc)) in enumerate(results.items()):
        img_with_kp = visualize_keypoints(test_img, kp)
        img_rgb = cv2.cvtColor(img_with_kp, cv2.COLOR_BGR2RGB)
        
        axes[idx].imshow(img_rgb)
        axes[idx].set_title(f'{detector_name.upper()}\n{len(kp)} keypoints', fontsize=14)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('../results/figures/02_detector_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Resumen estadístico
    print("\nRESUMEN DE DETECTORES:")
    print("-" * 60)
    for detector_name, (kp, desc) in results.items():
        if desc is not None:
            print(f"\n{detector_name.upper()}:")
            print(f"  - Keypoints detectados: {len(kp)}")
            print(f"  - Dimensión descriptor: {desc.shape[1]}")
            print(f"  - Tipo descriptor: {desc.dtype}")

## 5. Emparejamiento de Características

Si hay al menos 2 imágenes, probamos el emparejamiento.

In [ ]:
if len(images) >= 2:
    # Tomar dos imágenes
    name1, img1 = images[0]
    name2, img2 = images[1]
    
    print(f"\nEmparejando: {name1} <-> {name2}")
    print("="*60)
    
    # Detectar características con SIFT
    kp1, desc1 = detect_sift_features(img1)
    kp2, desc2 = detect_sift_features(img2)
    
    # Probar diferentes métodos de emparejamiento
    methods = ['flann', 'bf']
    
    for method in methods:
        print(f"\n{method.upper()}:")
        matches = match_features(desc1, desc2, method=method, ratio_test=0.75)
        
        # Estadísticas
        stats = compute_match_statistics(matches)
        print(f"  - Matches: {stats['num_matches']}")
        print(f"  - Distancia promedio: {stats['mean_distance']:.2f}")
        print(f"  - Distancia std: {stats['std_distance']:.2f}")
        
        # Visualizar
        img_matches = visualize_matches(img1, kp1, img2, kp2, matches, max_matches=50)
        img_matches_rgb = cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(15, 8))
        plt.imshow(img_matches_rgb)
        plt.title(f'Matches - {method.upper()}: {len(matches)} matches', fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'../results/figures/03_matches_{method}.png', dpi=150, bbox_inches='tight')
        plt.show()
else:
    print("⚠ Se necesitan al menos 2 imágenes para emparejar")

## 6. Conclusiones del Análisis Exploratorio

**Resumen:**

1. **Imágenes cargadas:** Se han cargado las imágenes del comedor exitosamente
2. **Detectores:** 
   - SIFT: Mejor para imágenes de alta calidad, más keypoints robustos
   - ORB: Más rápido, bueno para aplicaciones en tiempo real
   - AKAZE: Balance entre velocidad y precisión
3. **Emparejamiento:**
   - FLANN es más rápido para conjuntos grandes
   - BFMatcher es más preciso pero más lento
4. **Recomendación:** Usar SIFT con FLANN para el registro final

**Próximos pasos:**
- Validar con imágenes sintéticas (Notebook 02)
- Implementar pipeline completo de registro (Notebook 03)

In [ ]:
print("\n" + "="*60)
print("ANÁLISIS EXPLORATORIO COMPLETADO")
print("="*60)
print("\n✓ Resultados guardados en: ../results/figures/")
print("\nContinúe con el Notebook 02 para validación sintética.")